# Data Download - opcode usage by block

#### Maria Silva and Shouqiao Wang

#### March 2025

## 1. Imports and directories

In [ ]:
import os
import sys
import json
import pandas as pd

In [ ]:
# Main directories and files
current_path = os.getcwd()
repo_dir = os.path.abspath(os.path.join(current_path, ".."))
data_dir = os.path.join(repo_dir, "data")
src_dir = os.path.join(repo_dir, "src")

In [ ]:
# import internal packages
sys.path.append(src_dir)
from data import get_opcode_gas_for_block

## 2. Setup

In [ ]:
# Secrets for acessing xatu clickhouse and erigon
with open(os.path.join(repo_dir, "secrets.json"), "r") as file:
    secrets_dict = json.load(file)

# Block heights
block_start = 22000000  # Mar-08-2025
block_count = 6000 # ~1 day of ETH blocks

## 3. Download and save data

In [ ]:
df = pd.DataFrame()
for block_height in range(block_start, block_start + block_count):
    block_df = get_opcode_gas_for_block(
        block_height,
        secrets_dict["xatu_username"],
        secrets_dict["xatu_password"],
        secrets_dict["erigon_username"],
        secrets_dict["erigon_password"],
    )
    df = pd.concat([df, block_df], ignore_index=True)
    if block_height%1000==999: # save and reset
      out_file = os.path.join(data_dir, f"opcode_gas_usage_{block_height-999}_{block_height}.csv")
      df.to_csv(out_file, index=False)
      df = pd.DataFrame()
    elif block_height%50==0: # checkpoint
      temp_file = os.path.join(data_dir, f"opcode_gas_usage_temp.csv")
      df.to_csv(temp_file, index=False)

# Save last loop, in case of block counts not divisible by 1000
out_file = os.path.join(data_dir, f"opcode_gas_usage_end.csv")
df.to_csv(out_file, index=False)

In [ ]:
df.info()